# Project 32: Container Network Traffic Pattern Analysis

**Objective:** Build a machine learning model that can classify the type of application running inside a container (e.g., 'WebApp', 'Database', 'Cache') by analyzing the statistical features of its network traffic.

**Dataset Source:** Synthetically Generated (simulated network flow data from different containerized applications)

**Model:** RandomForestClassifier for learning complex patterns that differentiate network behavior of various applications

**Instructions:**
This notebook is fully self-contained and does not require external files. Simply run all cells in sequence.

In [ ]:
# ==================================================================================
#  Project 32: Container Network Traffic Analysis - Setup and Imports
# ==================================================================================

import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
import time

# Set random seeds for reproducibility
random.seed(42)
np.random.seed(42)

print("All libraries imported successfully.")

In [ ]:
# ==================================================================================
#  Synthetic Container Traffic Data Generation
# ==================================================================================

print("--- Generating Synthetic Container Network Traffic Dataset ---")

num_samples = 5000
data = []
app_types = ['WebApp', 'Database', 'Cache', 'MessageQueue', 'APIGateway']

# Define the "network personality" of each application
app_profiles = {
    'WebApp':       {'avg_pkt_size': 500,  'server_port': 443,  'flow_duration_ms': 500, 'client_server_ratio': 0.8},
    'Database':     {'avg_pkt_size': 1000, 'server_port': 5432, 'flow_duration_ms': 100, 'client_server_ratio': 0.5},
    'Cache':        {'avg_pkt_size': 150,  'server_port': 6379, 'flow_duration_ms': 20,  'client_server_ratio': 0.5},
    'MessageQueue': {'avg_pkt_size': 300,  'server_port': 5672, 'flow_duration_ms': 10000,'client_server_ratio': 0.5},
    'APIGateway':   {'avg_pkt_size': 800,  'server_port': 8080, 'flow_duration_ms': 200, 'client_server_ratio': 0.7}
}

print("\nApplication Network Profiles:")
for app, profile in app_profiles.items():
    print(f"{app:12}: avg_pkt={profile['avg_pkt_size']:4}B, port={profile['server_port']:4}, duration={profile['flow_duration_ms']:5}ms, ratio={profile['client_server_ratio']:.1f}")

for _ in range(num_samples):
    app_type = random.choice(app_types)
    profile = app_profiles[app_type]
    
    # Generate features based on the profile with some randomness
    avg_pkt_size = max(50, np.random.normal(profile['avg_pkt_size'], 50))
    server_port = profile['server_port']
    flow_duration_ms = max(1, np.random.normal(profile['flow_duration_ms'], 100))
    # Ratio of packets sent by client vs. server
    client_server_ratio = np.clip(np.random.normal(profile['client_server_ratio'], 0.1), 0.1, 0.9)
    
    # Number of packets in the flow
    total_packets = np.random.randint(5, 100)
    client_packets = int(total_packets * client_server_ratio)
    server_packets = total_packets - client_packets
    
    # Additional derived features
    total_bytes = avg_pkt_size * total_packets
    throughput_kbps = (total_bytes * 8) / (flow_duration_ms / 1000) / 1000  # Kbps
    
    data.append([avg_pkt_size, server_port, flow_duration_ms, client_packets, 
                server_packets, total_bytes, throughput_kbps, app_type])

df = pd.DataFrame(data, columns=['avg_pkt_size', 'server_port', 'flow_duration_ms', 
                                'client_packets', 'server_packets', 'total_bytes', 
                                'throughput_kbps', 'app_type'])

# Remove any anomalous data
df = df[(df['flow_duration_ms'] > 0) & (df['throughput_kbps'] > 0)]

print(f"\nDataset generation complete. Created {len(df)} flow samples.")
print(f"Features: {list(df.columns[:-1])}")
print(f"Target: {df.columns[-1]}")

print("\nDataset Sample:")
print(df.sample(10).round(2))

print("\nApplication Type Distribution:")
print(df['app_type'].value_counts())

In [ ]:
# ==================================================================================
#  Data Exploration and Visualization
# ==================================================================================

print("--- Data Exploration and Pattern Analysis ---")

# Statistical summary
print("\nDataset Statistics:")
print(df.describe().round(2))

# Create comprehensive visualization
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Container Application Network Traffic Patterns', fontsize=16)

# 1. Average packet size by application type
sns.boxplot(data=df, x='app_type', y='avg_pkt_size', ax=axes[0,0])
axes[0,0].set_title('Average Packet Size by Application')
axes[0,0].tick_params(axis='x', rotation=45)

# 2. Flow duration by application type
sns.boxplot(data=df, x='app_type', y='flow_duration_ms', ax=axes[0,1])
axes[0,1].set_title('Flow Duration by Application')
axes[0,1].set_yscale('log')
axes[0,1].tick_params(axis='x', rotation=45)

# 3. Throughput by application type
sns.boxplot(data=df, x='app_type', y='throughput_kbps', ax=axes[0,2])
axes[0,2].set_title('Throughput by Application')
axes[0,2].set_yscale('log')
axes[0,2].tick_params(axis='x', rotation=45)

# 4. Client vs Server packets scatter
for app in app_types:
    app_data = df[df['app_type'] == app]
    axes[1,0].scatter(app_data['client_packets'], app_data['server_packets'], 
                     alpha=0.6, label=app)
axes[1,0].set_xlabel('Client Packets')
axes[1,0].set_ylabel('Server Packets')
axes[1,0].set_title('Client vs Server Packet Distribution')
axes[1,0].legend(bbox_to_anchor=(1.05, 1), loc='upper left')

# 5. Server port distribution
port_counts = df.groupby(['app_type', 'server_port']).size().unstack(fill_value=0)
port_counts.plot(kind='bar', stacked=True, ax=axes[1,1])
axes[1,1].set_title('Server Port Usage by Application')
axes[1,1].tick_params(axis='x', rotation=45)
axes[1,1].legend(title='Server Port', bbox_to_anchor=(1.05, 1), loc='upper left')

# 6. Correlation heatmap
correlation_matrix = df.select_dtypes(include=[np.number]).corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, ax=axes[1,2])
axes[1,2].set_title('Feature Correlation Matrix')

plt.tight_layout()
plt.show()

# Application profile analysis
print("\nApplication Profile Analysis:")
profile_stats = df.groupby('app_type').agg({
    'avg_pkt_size': ['mean', 'std'],
    'flow_duration_ms': ['mean', 'std'],
    'throughput_kbps': ['mean', 'std'],
    'server_port': 'first'
}).round(2)

profile_stats.columns = ['_'.join(col).strip() for col in profile_stats.columns]
print(profile_stats)

In [ ]:
# ==================================================================================
#  Data Preprocessing and Splitting
# ==================================================================================

print("--- Data Preprocessing and Splitting ---")

# Separate features and target
X = df.drop(columns=['app_type'])
y = df['app_type']

print(f"Feature matrix shape: {X.shape}")
print(f"Target vector shape: {y.shape}")
print(f"Features: {list(X.columns)}")

# Encode the string labels
le = LabelEncoder()
y_encoded = le.fit_transform(y)

print(f"\nLabel encoding:")
for i, class_name in enumerate(le.classes_):
    print(f"  {class_name}: {i}")

# Use a stratified split to ensure all app types are represented
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.3, random_state=42, stratify=y_encoded
)

print(f"\nTraining set: {X_train.shape}")
print(f"Test set: {X_test.shape}")

# Display class distribution in train/test sets
print("\nClass distribution in training set:")
train_dist = pd.Series(y_train).value_counts().sort_index()
for i, count in enumerate(train_dist):
    print(f"  {le.classes_[i]}: {count} samples")

print("\nClass distribution in test set:")
test_dist = pd.Series(y_test).value_counts().sort_index()
for i, count in enumerate(test_dist):
    print(f"  {le.classes_[i]}: {count} samples")

In [ ]:
# ==================================================================================
#  Model Training
# ==================================================================================

print("--- Model Training ---")

# Initialize RandomForest with optimized parameters
model = RandomForestClassifier(
    n_estimators=100, 
    random_state=42, 
    n_jobs=-1,
    max_depth=10,
    min_samples_split=5,
    min_samples_leaf=2
)

print("Training the RandomForestClassifier...")
start_time = time.time()
model.fit(X_train, y_train)
end_time = time.time()
print(f"Training completed in {end_time - start_time:.2f} seconds.")

# Display model parameters
print(f"\nModel Configuration:")
print(f"• Number of estimators: {model.n_estimators}")
print(f"• Max depth: {model.max_depth}")
print(f"• Min samples split: {model.min_samples_split}")
print(f"• Min samples leaf: {model.min_samples_leaf}")

In [ ]:
# ==================================================================================
#  Model Evaluation
# ==================================================================================

print("--- Model Evaluation ---")

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"\nModel Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")

# Detailed classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=le.classes_))

# Per-class accuracy analysis
print("\nPer-Class Performance Analysis:")
cm = confusion_matrix(y_test, y_pred)
for i, class_name in enumerate(le.classes_):
    class_accuracy = cm[i, i] / np.sum(cm[i, :])
    print(f"  {class_name:12}: {class_accuracy:.3f} ({class_accuracy*100:.1f}%)")

In [ ]:
# ==================================================================================
#  Results Visualization
# ==================================================================================

print("--- Results Visualization ---")

# Create comprehensive evaluation visualization
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Container Application Classification Results', fontsize=16)

# 1. Confusion Matrix
sns.heatmap(cm, annot=True, fmt='d', cmap='rocket', 
            xticklabels=le.classes_, yticklabels=le.classes_, ax=ax1)
ax1.set_title('Confusion Matrix for Container Application Classification')
ax1.set_ylabel('Actual Application')
ax1.set_xlabel('Predicted Application')

# 2. Feature Importance
importances = model.feature_importances_
features = X.columns
feature_importance_df = pd.DataFrame({
    'Feature': features, 
    'Importance': importances
}).sort_values('Importance', ascending=True)

ax2.barh(feature_importance_df['Feature'], feature_importance_df['Importance'])
ax2.set_title('Feature Importance in Container Traffic Classification')
ax2.set_xlabel('Importance')

# 3. Prediction confidence distribution
y_pred_proba = model.predict_proba(X_test)
max_probabilities = np.max(y_pred_proba, axis=1)
ax3.hist(max_probabilities, bins=30, alpha=0.7, color='green', edgecolor='black')
ax3.set_title('Prediction Confidence Distribution')
ax3.set_xlabel('Maximum Probability')
ax3.set_ylabel('Frequency')
ax3.axvline(np.mean(max_probabilities), color='red', linestyle='--', 
           label=f'Mean: {np.mean(max_probabilities):.3f}')
ax3.legend()

# 4. Application classification accuracy
class_accuracies = [cm[i, i] / np.sum(cm[i, :]) for i in range(len(le.classes_))]
ax4.bar(le.classes_, class_accuracies, color='skyblue', alpha=0.7)
ax4.set_title('Per-Application Classification Accuracy')
ax4.set_ylabel('Accuracy')
ax4.set_ylim(0, 1)
ax4.tick_params(axis='x', rotation=45)

# Add accuracy values on bars
for i, acc in enumerate(class_accuracies):
    ax4.text(i, acc + 0.01, f'{acc:.3f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

# Display feature importance table
print("\n--- Feature Importance: What defines a container's network personality? ---")
feature_importance_sorted = feature_importance_df.sort_values('Importance', ascending=False)
print(feature_importance_sorted.round(4))

In [ ]:
# ==================================================================================
#  Misclassification Analysis
# ==================================================================================

print("--- Misclassification Analysis ---")

# Find misclassified samples
misclassified_mask = (y_test != y_pred)
misclassified_indices = np.where(misclassified_mask)[0]

print(f"Total misclassified samples: {len(misclassified_indices)} out of {len(y_test)} ({len(misclassified_indices)/len(y_test)*100:.1f}%)")

if len(misclassified_indices) > 0:
    # Analyze misclassification patterns
    misclass_analysis = pd.DataFrame({
        'Actual': [le.classes_[y_test[i]] for i in misclassified_indices],
        'Predicted': [le.classes_[y_pred[i]] for i in misclassified_indices],
        'Confidence': [max_probabilities[i] for i in misclassified_indices]
    })
    
    print("\nMost common misclassification patterns:")
    misclass_patterns = misclass_analysis.groupby(['Actual', 'Predicted']).size().sort_values(ascending=False)
    print(misclass_patterns.head(10))
    
    print(f"\nAverage confidence in misclassified predictions: {misclass_analysis['Confidence'].mean():.3f}")
    print(f"Average confidence in correct predictions: {max_probabilities[~misclassified_mask].mean():.3f}")
    
    # Show some examples of misclassified samples
    print("\nExamples of misclassified samples:")
    X_test_reset = X_test.reset_index(drop=True)
    sample_indices = misclassified_indices[:5]  # Show first 5 misclassified samples
    
    for idx in sample_indices:
        actual_app = le.classes_[y_test[idx]]
        predicted_app = le.classes_[y_pred[idx]]
        confidence = max_probabilities[idx]
        print(f"\nSample {idx}:")
        print(f"  Actual: {actual_app}, Predicted: {predicted_app} (confidence: {confidence:.3f})")
        print(f"  Features: {dict(X_test_reset.iloc[idx].round(2))}")

# Calculate feature-based separation analysis
print("\n--- Feature Separation Analysis ---")
feature_stats = df.groupby('app_type')[['avg_pkt_size', 'flow_duration_ms', 'throughput_kbps']].agg(['mean', 'std'])
print("\nKey features by application type:")
print(feature_stats.round(2))

In [ ]:
# ==================================================================================
#  Application Prediction Examples
# ==================================================================================

print("--- Application Prediction Examples ---")

# Create example network flows for prediction
example_flows = [
    # WebApp: Small packets, HTTPS port, moderate duration
    {'avg_pkt_size': 480, 'server_port': 443, 'flow_duration_ms': 450, 
     'client_packets': 15, 'server_packets': 8, 'total_bytes': 11040, 'throughput_kbps': 196},
    
    # Database: Large packets, PostgreSQL port, short duration
    {'avg_pkt_size': 980, 'server_port': 5432, 'flow_duration_ms': 95, 
     'client_packets': 8, 'server_packets': 12, 'total_bytes': 19600, 'throughput_kbps': 1651},
    
    # Cache: Small packets, Redis port, very short duration
    {'avg_pkt_size': 140, 'server_port': 6379, 'flow_duration_ms': 18, 
     'client_packets': 5, 'server_packets': 5, 'total_bytes': 1400, 'throughput_kbps': 622},
    
    # MessageQueue: Medium packets, RabbitMQ port, long duration
    {'avg_pkt_size': 320, 'server_port': 5672, 'flow_duration_ms': 9500, 
     'client_packets': 12, 'server_packets': 13, 'total_bytes': 8000, 'throughput_kbps': 6.7},
    
    # APIGateway: Large packets, custom port, short duration
    {'avg_pkt_size': 750, 'server_port': 8080, 'flow_duration_ms': 180, 
     'client_packets': 14, 'server_packets': 6, 'total_bytes': 15000, 'throughput_kbps': 667}
]

# Convert to DataFrame
examples_df = pd.DataFrame(example_flows)

# Make predictions
predictions = model.predict(examples_df)
prediction_probabilities = model.predict_proba(examples_df)

print("\nContainer Application Predictions:")
print("=" * 100)

for i, (flow, pred_idx) in enumerate(zip(example_flows, predictions)):
    predicted_app = le.classes_[pred_idx]
    confidence = np.max(prediction_probabilities[i])
    
    print(f"\nExample {i+1}:")
    print(f"  Network Flow Characteristics:")
    print(f"    • Average packet size: {flow['avg_pkt_size']} bytes")
    print(f"    • Server port: {flow['server_port']}")
    print(f"    • Flow duration: {flow['flow_duration_ms']} ms")
    print(f"    • Client/Server packets: {flow['client_packets']}/{flow['server_packets']}")
    print(f"    • Total bytes: {flow['total_bytes']:,}")
    print(f"    • Throughput: {flow['throughput_kbps']:.1f} Kbps")
    
    print(f"  Prediction: {predicted_app} (confidence: {confidence:.3f})")
    
    # Show probability distribution
    print(f"  Probability distribution:")
    for j, app_name in enumerate(le.classes_):
        prob = prediction_probabilities[i][j]
        print(f"    {app_name:12}: {prob:.3f} {'█' * int(prob * 20)}")
    
    print("-" * 80)

In [ ]:
# ==================================================================================
#  Conclusion
# ==================================================================================

print("--- Conclusion ---")
print("The RandomForest model successfully learned to classify containerized applications based on their distinct network traffic patterns.")

print("\nKey Performance Results:")
print(f"• Overall accuracy: {accuracy:.3f} ({accuracy*100:.1f}%)")
print(f"• Total samples analyzed: {len(df):,}")
print(f"• Number of application types: {len(le.classes_)}")
print(f"• Training time: {end_time - start_time:.2f} seconds")
print(f"• Average prediction confidence: {np.mean(max_probabilities):.3f}")

print("\nMost Important Network Characteristics:")
top_features = feature_importance_sorted.head(3)
for idx, row in top_features.iterrows():
    print(f"• {row['Feature']}: {row['Importance']:.3f} importance")

print("\nBusiness Impact:")
print("• **Container Visibility**: Automatically identify application types without deep packet inspection")
print("• **Resource Optimization**: Right-size containers based on application network behavior")
print("• **Security Monitoring**: Detect anomalous applications or configuration drift")
print("• **Network Planning**: Optimize network policies based on application traffic patterns")

print("\nOperational Applications:")
print("• **Auto-Discovery**: Automatically classify unknown containers in production")
print("• **Performance Monitoring**: Baseline normal behavior for each application type")
print("• **Compliance**: Ensure containers match expected application profiles")
print("• **Anomaly Detection**: Flag containers with unexpected network behavior")

print("\nTechnical Insights:")
print("• Server port is highly predictive but not sufficient alone")
print("• Packet size and flow duration create distinct 'network personalities'")
print("• Client/server packet ratios reveal application communication patterns")
print("• Throughput characteristics distinguish between high and low bandwidth applications")

print("\nReal-world Deployment:")
print("• Integrate with container orchestration platforms (Kubernetes, Docker Swarm)")
print("• Deploy as sidecar containers for real-time classification")
print("• Use with network monitoring tools (Prometheus, Grafana) for dashboards")
print("• Combine with policy engines for automated container management")

print(f"\nModel Reliability:")
if len(misclassified_indices) > 0:
    print(f"• Misclassification rate: {len(misclassified_indices)/len(y_test)*100:.1f}%")
    print(f"• Most confused applications require additional feature engineering")
else:
    print("• Perfect classification on test set - model generalizes well")
print(f"• High prediction confidence indicates robust decision boundaries")
print(f"• Feature importance aligns with networking domain knowledge")